In [3]:
from ortools.constraint_solver import pywrapcp

# Creates the solver.
solver = pywrapcp.Solver("group_assignments")

num_students = 4
num_groups = 2

c =[[0 for k in range(num_students)] for i in range(num_students)]
c[0][1] = 1
c[1][0] = 1
c[2][3] = 1
c[3][2] = 1
c[2][1] = 1
c[1][2] = 1
c[3][0] = 1
c[0][3] = 1

In [4]:
x = {}
for i in range(num_students):
    for j in range(num_groups):
        x[(i, j)] = solver.IntVar(0, 1, "x(%i,%i)" % (i, j))
x_flat = [x[(i, j)] for i in range(num_students) for j in range(num_groups)]

# constraint one
for j in range(num_groups):
    solver.Add(solver.Sum([x[(i, j)] > 0 for i in range(num_students)]) >= 2) # at least one student per group
    solver.Add(solver.Sum([x[(i, j)] > 0 for i in range(num_students)]) <= 2) #num_students - num_groups + 1) # at most number of students per group

    # constraint two    
for i in range(num_students):
    solver.Add(solver.Sum([x[(i, j)] > 0 for j in range(num_groups)]) == 1) # exactly one group per student

# constraint on the historical co-existance of students     
for j in range(num_groups):
    solver.Add(solver.Sum([x[(i, j)] * x[(k, j)] * c[i][k] > 0 for k in range(num_students) for i in range(num_students)]) == 0)
    
# Create the decision builder.
db = solver.Phase(x_flat, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

# Create the solution collector.
solution = solver.Assignment()
solution.Add(x_flat)
collector = solver.AllSolutionCollector(solution)

solver.Solve(db, [collector])
num_solutions = collector.SolutionCount()
print("Solutions found:", num_solutions)
print("Time:", solver.WallTime(), "ms")
print()

# Display a few solutions picked at random.
a_few_solutions = [0]#, 1, 2, 3]

for sol in a_few_solutions:
    print("Solution number" , sol, '\n')
    for j in range(num_groups):
        for i in range(num_students):
            xij = collector.Value(sol, x[(i, j)])
            if (xij == 1):
                print("Student", i, "assigned to Group", j)
        print()

('Solutions found:', 2)
('Time:', 1493, 'ms')
()
('Solution number', 0, '\n')
('Student', 1, 'assigned to Group', 0)
('Student', 3, 'assigned to Group', 0)
()
('Student', 0, 'assigned to Group', 1)
('Student', 2, 'assigned to Group', 1)
()
